In [2]:
import csv
import glob
import os
import time
import traceback
from retry import retry

from binance import Client

import config as cf

@retry()
def run() -> bool:
    client = Client(cf.API_KEY, cf.API_SECRET)

    tickers_of_interest = ['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'EOSUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'VETUSDT', 'XRPUSDT']
    tickers = [ticker for ticker in client.get_all_tickers() if ticker['symbol'] in tickers_of_interest]

    intervals = [
        Client.KLINE_INTERVAL_1MINUTE,
        Client.KLINE_INTERVAL_3MINUTE,
        Client.KLINE_INTERVAL_5MINUTE,
        Client.KLINE_INTERVAL_15MINUTE,
        Client.KLINE_INTERVAL_30MINUTE,
        Client.KLINE_INTERVAL_1HOUR,
        Client.KLINE_INTERVAL_2HOUR,
        Client.KLINE_INTERVAL_4HOUR,
        Client.KLINE_INTERVAL_6HOUR,
        Client.KLINE_INTERVAL_8HOUR,
        Client.KLINE_INTERVAL_12HOUR,
        Client.KLINE_INTERVAL_1DAY,
        Client.KLINE_INTERVAL_3DAY,
        Client.KLINE_INTERVAL_1WEEK,
    ]

    success = True
    skip = False
    files = []
    intervals.reverse()
    stats = open("stats.csv", 'w', newline='')
    stats_writer = csv.writer(stats, delimiter=',')
    acc_time = 0
    path = ""
    try:
        for interval in intervals:
            files = []
            for file in glob.glob("./data/" + interval + "/*.csv"):
                files.append(os.path.basename(file))
            stats_writer.writerow(
                ["\t\tNEW INTERVAL -> " + interval + " Elapsed time: " + str(acc_time)])
            print("\t\tNEW INTERVAL -> " + interval +
                  " Elapsed time: " + str(acc_time))

            for ticker in tickers:
                start = time.time()
                if ticker["symbol"] + ".csv" not in files:
                    stats_writer.writerow(
                        ["TICKER STARTED -> " + ticker["symbol"] + interval])
                    print("TICKER STARTED -> " + ticker["symbol"] + interval)
                    path = "./data/" + interval + "/" + ticker['symbol'] + ".csv"
                    file = open(path, 'w', newline='')
                    path = file.name
                    writer = csv.writer(file, delimiter=',')
                    candles = client.get_historical_klines(
                        ticker['symbol'], interval, '1 Jan, 2017', '1 day ago UTC')
                    for candle in candles:
                        writer.writerow(candle)

                    et = time.time() - start
                    acc_time += et
                    stats_writer.writerow(
                        ["TICKER FINISHED -> " + ticker["symbol"] + interval + " Elapsed time: " + str(et)])
                    print("TICKER FINISHED -> " +
                          ticker["symbol"] + interval + " Elapsed time: " + str(et))
                else:
                    et = time.time() - start
                    acc_time += et
                    print("TICKER SKIPPED -> " +
                          ticker["symbol"] + interval + " Elapsed time: " + str(et))
                    stats_writer.writerow(
                        ["TICKER SKIPPED -> " + ticker["symbol"] + interval + " Elapsed time: " + str(et)])

    except Exception as e:
        success = False
        traceback.print_exc()

    finally:

        if success:
            final_str = "----------------------SUCCESS----------------------\n\t\t Total Time: " + \
                str(acc_time)
        else:
            final_str = "----------------------ERROR----------------------\n\t\t Total Time: " + \
                str(acc_time)
            
        print(final_str)
        stats_writer.writerow([final_str])
        #if not success:
        #file.close()
        #os.remove(path)
        if not success:
            stats_writer.writerow(["------------------------Retry-------------------------"])
            raise Exception("Something happened - Retrying")
        return success

run()

		NEW INTERVAL -> 1w Elapsed time: 0
TICKER SKIPPED -> BTCUSDT1w Elapsed time: 7.152557373046875e-07
TICKER SKIPPED -> ETHUSDT1w Elapsed time: 4.76837158203125e-07
TICKER SKIPPED -> BNBUSDT1w Elapsed time: 7.152557373046875e-07
TICKER SKIPPED -> LTCUSDT1w Elapsed time: 4.76837158203125e-07
TICKER SKIPPED -> ADAUSDT1w Elapsed time: 4.76837158203125e-07
TICKER SKIPPED -> XRPUSDT1w Elapsed time: 2.384185791015625e-07
TICKER SKIPPED -> EOSUSDT1w Elapsed time: 2.384185791015625e-07
TICKER SKIPPED -> TRXUSDT1w Elapsed time: 0.0
TICKER SKIPPED -> VETUSDT1w Elapsed time: 2.384185791015625e-07
		NEW INTERVAL -> 3d Elapsed time: 3.5762786865234375e-06
TICKER SKIPPED -> BTCUSDT3d Elapsed time: 9.5367431640625e-07
TICKER SKIPPED -> ETHUSDT3d Elapsed time: 0.0
TICKER SKIPPED -> BNBUSDT3d Elapsed time: 2.384185791015625e-07
TICKER SKIPPED -> LTCUSDT3d Elapsed time: 2.384185791015625e-07
TICKER SKIPPED -> ADAUSDT3d Elapsed time: 0.0
TICKER SKIPPED -> XRPUSDT3d Elapsed time: 2.384185791015625e-07
TICK

True